In [ ]:
from qiskit_nature.second_q.drivers import ElectronicStructureDriver, PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
# https://www.webqc.org/molecularformatsconverter.php

In [ ]:
# https://docs.python.org/3/tutorial/inputoutput.html
with open("DataBank Folder/menthol_tester.xyz") as f:
    s = f.readlines()
xyz = s[2:]
print(xyz)
def process_xyz(file):
    total = []
    symbols = [i.split()[0] for i in file]
    total = [tuple(i.split()[1:]) for i in file]
    return symbols, total
molecule = MoleculeInfo(
    symbols = process_xyz(xyz)[0],
    coords = process_xyz(xyz)[1],
    multiplicity=1,
    charge=0
)


In [ ]:
from qiskit_nature.second_q.problems import ElectronicStructureProblem
driver = PySCFDriver.from_molecule(molecule, basis="sto3g")
problem = driver.run()

In [ ]:
from qiskit_nature.second_q.drivers import PySCFDriver
driver = PySCFDriver(
    atom="; ".join([i.replace('\n', ' ') for i in xyz]),
    unit = DistanceUnit.ANGSTROM, 
)
problem = driver.run()

In [ ]:
"; ".join([i.replace('\n', ' ') for i in xyz])

In [ ]:
problem.hamiltonian.electronic_integrals.register_length
problem.hamiltonian

In [ ]:
second_qs = problem.second_q_ops()

In [ ]:
#https://github.com/kjwcoo/Qiskit_Hackathon_Korea_2022_Nature/blob/main/9037647705611511190qiskit_nature_challenge.ipynb
#https://github.com/qiskit-community/qiskit-application-modules-demo-sessions/blob/main/qiskit-nature/Qiskit_Nature_YouTube_Tutorial.ipynb

In [ ]:
Insights

# Insights
Electronic Structure (Kinetic, Potential, Coulomb + Exchange)
Born Oppenheimer -> Separate e motion from nuclei
Non-rel TI Schro Eq -> Use Variational Principle
Second Quantization (Map to Spin / Qubits)
Hamiltonian -> Creation + Annihilation (Excite + De-excite) -> Occupation Number Vector

Problem -> Solver -> Result
Problem (Driver -> Properties tfed by Transformers ) -> Ferm Op

Op -> Solver (Converter -> Operator + Eigensolver) -> Result

In [ ]:
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
transformer = ActiveSpaceTransformer(
    num_electrons = 4,
    num_spatial_orbitals = 4,
)
#transformer.prepare_active_space(problem.num_particles, problem.num_spatial_orbitals)
#reduced_ham = transformer.transform_hamiltonian(problem.hamiltonian)
red_problem  = transformer.transform(problem)
#red_problem = ElectronicStructureProblem(driver, [transformer])

In [ ]:
#new_problem = ElectronicStructureProblem(reduced_ham)
print(red_problem.num_particles)

In [ ]:
#new_problem = ElectronicStructureProblem(reduced_ham)
#print(new_problem.second_q_ops())
reduced_ham = red_problem.second_q_ops()[0]
from qiskit_nature.second_q.mappers import JordanWignerMapper
mapper = JordanWignerMapper()
qubit_ham = mapper.map(reduced_ham)

In [ ]:
print(qubit_ham)

In [ ]:
print(red_problem.num_particles)

In [ ]:
print(red_problem.properties)
from qiskit.algorithms.optimizers import SLSQP
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
#from qiskit.providers.aer import StatevectorSimulator, QasmSimulator
from qiskit_nature.second_q.algorithms.initial_points import HFInitialPoint
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

ansatz = UCCSD(
    red_problem.num_spatial_orbitals,
    red_problem.num_particles,
    mapper,
    initial_state = HartreeFock(
        red_problem.num_spatial_orbitals,
        red_problem.num_particles,
        mapper
    )
)
initial_point = HFInitialPoint()
initial_point.ansatz = ansatz
initial_point.problem = red_problem
estimator = Estimator()
solver = VQE(estimator, ansatz, SLSQP())
solver.initial_point = initial_point.to_numpy_array()

In [ ]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
gr = GroundStateEigensolver(mapper, solver)
results = gr.solve(red_problem)

In [ ]:
print(results)
with open("MentholResults.txt", "w") as f:
    f.write(str(results))